# Explore Loading CAA geoTIFF

Exploring issue that Mélanie is having with loading bathymetry of the Canadian Arctic Archepelago
from a geoTIFF she downloaded from GMRT.

Using a conda env described by `environment.yaml`.
Exact package versions are recorded in `requirements.txt`.

In [1]:
%matplotlib inline

## Mélanie's `basemap` Code

In [ ]:
import os
# PROJ_LIB envvar is already defined in my env
# os.environ['PROJ_LIB'] = r'C://Users/Mel/Anaconda3/pkgs/proj4-5.2.0-h6538335_1006\Library\share'
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import osr
import numpy as np
import math

#Plot setup
fig= plt.figure(figsize=(10,8))

ax = plt.subplot(111,aspect = 'equal')
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0, hspace=0)

#Map setup
xx=1000
map = Basemap(resolution='c', projection='ortho', llcrnrx=-25*xx, llcrnry=-10*xx, urcrnrx=+15*xx, urcrnry=+15*xx, lon_0=-100, lat_0=75)
#map.drawcoastlines()
#map.drawcountries()
#map.fillcontinents(color='grey')

# parallels = np.arange(65,90,5)
# meridians = np.arange(-175,-50,20)

#Load colormap and setup elevation contour levels
cmap=plt.get_cmap("jet")
clevs = np.linspace(-4500, 0, 16)

# Load GeoTiff data
raster = 'C://Users/Mel/Documents/Recherche/Post-doc/Vancouver-Toulouse/Vancouver/PaTh_interpretation/CAA-MelvilleSound/GMRTv3_7_20200504topo.tif'

ds = gdal.Open(raster)

#Get the dimentions of column and row
nc   = ds.RasterXSize
nr   = ds.RasterYSize

#Read elevation data
data = ds.ReadAsArray()

#Get Longitude and Latitude info
geotransform = ds.GetGeoTransform()
xOrigin      = geotransform[0]
yOrigin      = geotransform[3]
pixelWidth   = geotransform[1]
pixelHeight  = geotransform[5]

#Generate Longitude and Latitude array
lons = xOrigin + np.arange(0, nc)*pixelWidth
lats = yOrigin + np.arange(0, nr)*pixelHeight

#Contour plot
x, y = map(*np.meshgrid(lons, lats))
##x, y = map(globals()[lons],globals()[lats])
cs=map.contourf(x, y, data, clevs, cmap=cmap)

map.drawparallels(parallels,labels=[1,0,0,0],color='k', fontsize=10, fontweight='bold')
map.drawmeridians(meridians,labels=[0,0,0,1],color='k', fontsize=10, fontweight='bold')

cb = map.colorbar(cs, 'bottom', size='5%', pad='10%')

cb.set_label('Elevation (m)', fontsize=12, fontweight='bold')
cb.ax.tick_params(labelsize=10)

plt.show()

This code crashes my notebook kernel.
I was able to trace that down to the use of the `ortho` projection.
As simplified demonstration:

In [2]:
from mpl_toolkits.basemap import Basemap

In [3]:
# works fine
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution='c',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)

In [ ]:
# crashes notebook kernel
m = Basemap(projection='ortho',
            resolution='c',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)

## Mélanie's `cartopy` Code

In [1]:
from osgeo import gdal, osr

gdal.UseExceptions()


# fname = 'C://Users/Mel/Documents/Recherche/Post-doc/Vancouver-Toulouse/Vancouver/PaTh_interpretation/CAA-MelvilleSound/GMRTv3_7_20200504topo.tif'
fname = "/home/doug/Downloads/GMRTv3_7_20200504topo.tif"

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)


import cartopy.crs as ccrs

projcs = inproj.GetAuthorityCode('LOCAL_CS')
projection = ccrs.epsg(projcs)
print(projection)

import matplotlib.pyplot as plt

subplot_kw = dict(projection=projection)
fig, ax = plt.subplots(figsize=(9, 9), subplot_kw=subplot_kw)

extent = (gt[0], gt[0] + ds.RasterXSize * gt[1],
          gt[3] + ds.RasterYSize * gt[5], gt[3])

img = ax.imshow(data[:3, :, :].transpose((1, 2, 0)), extent=extent,
                origin='upper')

# create a grid of xy coordinates in the original projection
xy_source = np.mgrid[xmin:xmax+xres:xres, ymax+yres:ymin:yres]

# Convert from source projection to basemap projection
xx, yy = convertXY(xy_source, inproj, outproj)

# plot the data (first layer)
im1 = m.pcolormesh(xx, yy, data[0,:,:].T, cmap=plt.cm.jet)

PROJCS["WGS 84 / Arctic Polar Stereographic",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Polar_Stereographic"],
    PARAMETER["latitude_of_origin",71],
    PARAMETER["central_meridian",0],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",SOUTH],
    AXIS["Northing",SOUTH],
    AUTHORITY["EPSG","3995"]]


ParseError: mismatched tag: line 34, column 4 (<string>)

The above traceback tells me that there is a problem in the geoTIFF file.
The only thing I can suggest is to ask GMRT about this.